In [3]:
import requests
import time

def plausibility(subject, obj, sleep=0.1, verbose=False):
    """
    Compute a plausibility score between two concepts using the ConceptNet API.
    Includes error handling, retry, and graceful fallback.
    """
    def normalize(concept):
        return concept.strip().lower().replace(" ", "_")

    s_norm = normalize(subject)
    o_norm = normalize(obj)

    def query_conceptnet(subj, obj):
        url = f"http://api.conceptnet.io/query?node=/c/en/{subj}&other=/c/en/{obj}"
        try:
            response = requests.get(url, timeout=10)
            if response.status_code != 200:
                if verbose:
                    print(f"⚠️ API returned status {response.status_code} for {subj}-{obj}")
                return []
            data = response.json()
            return data.get("edges", [])
        except requests.exceptions.RequestException as e:
            if verbose:
                print(f"⚠️ Request failed for {subj}-{obj}: {e}")
            return []
        except ValueError:
            # JSON decode failed
            if verbose:
                print(f"⚠️ JSON decode failed for {subj}-{obj}")
            return []

    # Try forward and reverse directions
    edges = query_conceptnet(s_norm, o_norm)
    if not edges:
        time.sleep(sleep)
        edges = query_conceptnet(o_norm, s_norm)

    if not edges:
        return 0.1  # default low plausibility if no valid edges found

    # Relation-type weighting
    relation_weights = {
        "/r/RelatedTo": 1.0,
        "/r/IsA": 0.9,
        "/r/UsedFor": 0.9,
        "/r/PartOf": 0.8,
        "/r/AtLocation": 0.7,
        "/r/CapableOf": 0.7,
        "/r/HasA": 0.7,
        "/r/MannerOf": 0.6,
        "/r/SimilarTo": 0.6,
        "/r/Antonym": 0.2,
    }

    scores = []
    for e in edges:
        rel = e.get("rel", {}).get("@id", "")
        weight = relation_weights.get(rel, 0.5)
        conceptnet_weight = e.get("weight", 1.0)
        scores.append(weight * conceptnet_weight)

    if verbose:
        print(f"✅ Found {len(edges)} edges between '{subject}' and '{obj}'")

    return round(sum(scores) / len(scores), 3)


In [4]:
pairs = [
    ("motorcyclist", "cigarette"),
    ("motorcyclist", "toothpick"),
    ("motorcyclist", "popsicle stick"),
    ("motorcyclist", "food"),
]

for s, o in pairs:
    print(f"{s} - {o}:", plausibility(s, o))


motorcyclist - cigarette: 0.1
motorcyclist - toothpick: 0.1
motorcyclist - popsicle stick: 0.1
motorcyclist - food: 0.1


In [7]:
import requests
import time

def plausibility_expanded(subject, obj, verbose=False, max_retries=3, sleep=0.2):
    """
    Compute plausibility using ConceptNet's /related endpoint.
    Includes error handling and fallback when ConceptNet returns no JSON.
    """
    def normalize(c):
        return c.strip().lower().replace(" ", "_")
    
    s_norm = normalize(subject)
    o_norm = normalize(obj)
    url = f"http://api.conceptnet.io/related/c/en/{s_norm}?filter=/c/en/{o_norm}"

    # Retry logic for reliability
    for attempt in range(max_retries):
        try:
            r = requests.get(url, timeout=10)
            if r.status_code != 200:
                if verbose:
                    print(f"⚠️ Status {r.status_code} for {s_norm}-{o_norm}")
                time.sleep(sleep)
                continue

            text = r.text.strip()
            if not text.startswith("{"):
                # Not JSON (HTML, empty, etc.)
                if verbose:
                    print(f"⚠️ Non-JSON response for {s_norm}-{o_norm}")
                time.sleep(sleep)
                continue

            data = r.json()
            related = data.get("related", [])
            if related:
                weight = related[0].get("weight", 0)
                if verbose:
                    print(f"🔗 Relatedness({subject}, {obj}) = {weight:.3f}")
                # Normalize roughly to [0,1]
                return round(min(weight / 30, 1.0), 3)
            else:
                if verbose:
                    print(f"ℹ️ No related terms for {subject}-{obj}")
                return 0.1

        except Exception as e:
            if verbose:
                print(f"⚠️ Attempt {attempt+1} failed for {subject}-{obj}: {e}")
            time.sleep(sleep)

    # Default fallback
    return 0.1


In [8]:
pairs = [
    ("motorcyclist", "cigarette"),
    ("motorcyclist", "toothpick"),
    ("motorcyclist", "popsicle stick"),
    ("motorcyclist", "food"),
]

for s, o in pairs:
    print(f"{s} - {o}: {plausibility_expanded(s, o, verbose=True)}")


⚠️ Status 502 for motorcyclist-cigarette
⚠️ Status 502 for motorcyclist-cigarette
⚠️ Status 502 for motorcyclist-cigarette
motorcyclist - cigarette: 0.1
⚠️ Status 502 for motorcyclist-toothpick
⚠️ Status 502 for motorcyclist-toothpick
⚠️ Status 502 for motorcyclist-toothpick
motorcyclist - toothpick: 0.1
⚠️ Status 502 for motorcyclist-popsicle_stick
⚠️ Status 502 for motorcyclist-popsicle_stick
⚠️ Status 502 for motorcyclist-popsicle_stick
motorcyclist - popsicle stick: 0.1
⚠️ Status 502 for motorcyclist-food
⚠️ Status 502 for motorcyclist-food
⚠️ Status 502 for motorcyclist-food
motorcyclist - food: 0.1
